In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from matplotlib.ticker import PercentFormatter
from scipy.stats import zscore


In [14]:
df=pd.read_excel(r"C:\Users\suhas\Downloads\marketing_campaign.xlsx")


In [15]:
df['Income'].fillna(df['Income'].mean(), inplace=True)


In [16]:
current_year = 2024 

# Calculate the age based on the birth year and the current year
df['Age'] = current_year - df['Year_Birth']

# Drop the original 'Year_Birth' column
df.drop(columns=['Year_Birth'], inplace=True)

# Save the modified DataFrame to a CSV file
df.to_csv('modified_data.csv', index=False)



In [17]:
from scipy.stats import zscore

# Select columns for outlier removal excluding 'Age'
columns_to_remove_outliers = ['NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
                               'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
                               'MntSweetProducts', 'MntGoldProds','Age' ,'Income']

# Calculate z-scores for selected columns
z_scores = zscore(df[columns_to_remove_outliers])

# Define a threshold for z-scores (e.g., 3 or -3, based on your preference)
z_score_threshold = 3

# Remove rows with outliers based on z-scores
df_no_outliers = df[(abs(z_scores) < z_score_threshold).all(axis=1)]

# Print the shape of the DataFrame after removing outliers
print("Shape of DataFrame after removing outliers:", df_no_outliers.shape)
df1=df_no_outliers


Shape of DataFrame after removing outliers: (1983, 29)


In [18]:
from datetime import datetime


# Feature 1: Total Spending
df1['Total_Spending'] = df1[['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']].sum(axis=1)

# Feature 2: Number of Children
df1['Num_Children'] = df1['Kidhome'] + df1['Teenhome']



# Feature 4: Accepted Campaigns Total
campaign_columns = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
df1['Total_Accepted_Campaigns'] = df1[campaign_columns].sum(axis=1)


#Feature5: Total Purchase
num_purchases_columns = ['NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases']
df1['TotalPurchases'] = df1[num_purchases_columns].sum(axis=1)


# Drop unnecessary columns
df1.drop(['Dt_Customer','Z_CostContact','Z_Revenue','Kidhome','Teenhome','ID','Recency','Complain'], axis=1, inplace=True)

# Save the modified DataFrame to a new CSV file
df1.to_csv('modified_data_with_features.csv', index=False)



C:\Users\suhas\AppData\Local\Temp\ipykernel_10204\1664692070.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Total_Spending'] = df1[['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']].sum(axis=1)
C:\Users\suhas\AppData\Local\Temp\ipykernel_10204\1664692070.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Num_Children'] = df1['Kidhome'] + df1['Teenhome']
C:\Users\suhas\AppData\Local\Temp\ipykernel_10204\1664692070.py:14: SettingWithCopyWarning

In [19]:
import streamlit as st
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Load your dataset here (replace 'modified_data_with_features.csv' with the actual filename)
df1 = pd.read_csv('modified_data_with_features.csv')

# Define the target variable and selected features
target_variable = 'Response'
selected_features = ['Total_Spending', 'Num_Children', 'Total_Accepted_Campaigns', 'TotalPurchases']
X = df1[selected_features]
y = df1[target_variable]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Level 1 Model: Predict whether a customer responds to any campaign (Binary Classification)
model_level1 = DecisionTreeClassifier(random_state=42)
model_level1.fit(X_train, y_train)

# Level 2 Model: Train a model for each campaign
response_columns = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
model_level2_dict = {}  # Dictionary to store Level 2 models

for response_column in response_columns:
    # Filter DataFrame for the selected campaign and samples where the customer responded
    df1_level2_train = df1.query(f'{response_column} == 1 and {target_variable} == 1')

    # Extract features and target variable
    X_train_level2 = df1_level2_train[selected_features]
    y_train_level2 = df1_level2_train[response_column]

    # Train the Level 2 model
    model_level2 = DecisionTreeClassifier(random_state=42)
    model_level2.fit(X_train_level2, y_train_level2)

    # Store the trained model in the dictionary
    model_level2_dict[response_column] = model_level2

# Streamlit app
st.title('Campaign Response Prediction App')

# Sidebar for user input
st.sidebar.header('User Input')
total_spending = st.sidebar.number_input('Total Spending', min_value=0)
num_children = st.sidebar.number_input('Number of Children', min_value=0)
total_campaigns = st.sidebar.number_input('Total Accepted Campaigns', min_value=0)
total_purchases = st.sidebar.number_input('Total Purchases', min_value=0)

# User input as a DataFrame
input_data = pd.DataFrame({
    'Total_Spending': [total_spending],
    'Num_Children': [num_children],
    'Total_Accepted_Campaigns': [total_campaigns],
    'TotalPurchases': [total_purchases]
})

# Level 1 Prediction
prediction_level1 = model_level1.predict(input_data)[0]

# Display Level 1 Prediction
st.subheader('Level 1 Prediction')
st.write(f'Customer Response Prediction: {prediction_level1}')

# Level 2 Prediction (if the customer responds)
if prediction_level1 == 1:
    # Iterate through each campaign
    for response_column in response_columns:
        # Filter DataFrame for the selected campaign and samples where the customer responded
        df1_level2_input = df1.query(f'{response_column} == 1 and {target_variable} == 1')

        # Extract features
        X_level2_input = df1_level2_input[selected_features]

        # Level 2 Prediction using the corresponding model from the dictionary
        model_level2 = model_level2_dict[response_column]
        predictions_level2 = model_level2.predict(X_level2_input)

        # Display Level 2 Prediction
        st.subheader(f'Level 2 Prediction for {response_column}')
        st.write(f'Customer Campaign Predictions: {", ".join(map(str, predictions_level2))}')
